Based on [ML data wrangling tutorial](https://elitedatascience.com/python-data-wrangling-tutorial)


## Goal: Implement a basic momentum trading strategy

### Rules:
- Buy the cryptocurrency that had the largest price gain over the previous week or month.
- Hold for exactly 7 days
- Sell our position
- Repeat for six months

In [2]:
import pandas as pd

pd.options.display.float_format = '{:,.2f}'.format
pd.options.display.max_rows = 200
pd.options.display.max_columns = 100

### Import the dataset

In [3]:
df = pd.read_csv('BNC2_sample.csv',
                 names=['Code', 'Date', 'Open', 'High', 'Low', 
                        'Close', 'Volume', 'VWAP', 'TWAP'])

In [4]:
df.head(5)

Code        Date   Open   High    Low  Close     Volume   VWAP   TWAP
0  GWA_BTC  2014-04-01 467.28 488.62 467.28 479.56  74,776.48 482.76 482.82
1  GWA_BTC  2014-04-02 479.20 494.30 431.32 437.08 114,052.96 460.19 465.93
2  GWA_BTC  2014-04-03 437.33 449.74 414.41 445.60  91,415.08 432.29 433.28
3  GWA_BTC  2014-04-04 445.18 456.10 429.16 449.81  51,147.27 443.46 443.93
4  GWA_BTC  2014-04-05 450.08 464.09 445.16 461.70  28,449.19 452.53 452.95

### Data Dictionary (for code GWA_BTC):

Date: The day on which the index values were calculated.

Open: The day's opening price index for Bitcoin in US dollars.

High: The highest value for the price index for Bitcoin in US dollars that day.

Low: The lowest value for the price index for Bitcoin in US dollars that day.

Close: The day's closing price index for Bitcoin in US dollars.

Volume: The volume of Bitcoin traded that day.

VWAP: The volume weighted average price of Bitcoin traded that day.

TWAP: The time-weighted average price of Bitcoin traded that day

### Inspect the data

In [5]:
print( df.Code.unique() )

['GWA_BTC' 'GWA_ETH' 'GWA_LTC' 'GWA_XLM' 'GWA_XRP' 'MWA_BTC_CNY'
 'MWA_BTC_EUR' 'MWA_BTC_GBP' 'MWA_BTC_JPY' 'MWA_BTC_USD' 'MWA_ETH_CNY'
 'MWA_ETH_EUR' 'MWA_ETH_GBP' 'MWA_ETH_JPY' 'MWA_ETH_USD' 'MWA_LTC_CNY'
 'MWA_LTC_EUR' 'MWA_LTC_GBP' 'MWA_LTC_JPY' 'MWA_LTC_USD' 'MWA_XLM_CNY'
 'MWA_XLM_EUR' 'MWA_XLM_USD' 'MWA_XRP_CNY' 'MWA_XRP_EUR' 'MWA_XRP_GBP'
 'MWA_XRP_JPY' 'MWA_XRP_USD']


In [6]:
df[df.Code.isin(['GWA_BTC', 'MWA_BTC_JPY', 'MWA_BTC_EUR']) 
   & (df.Date == '2018-01-01')]

Code        Date         Open         High          Low  \
1371       GWA_BTC  2018-01-01    14,505.89    14,505.89    13,617.46   
9074   MWA_BTC_EUR  2018-01-01    11,859.35    11,859.35    11,111.07   
11838  MWA_BTC_JPY  2018-01-01 1,674,341.45 1,678,567.55 1,572,173.90   

             Close     Volume         VWAP         TWAP  
1371     14,092.74 225,906.21    14,103.18    14,093.73  
9074     11,403.92  14,933.73    11,488.45    11,478.08  
11838 1,632,657.51  68,611.95 1,632,994.40 1,631,407.66

### Filter out unneeded records

Discard MWA but keep the GWA aggregate codes

In [7]:
gwa_codes = [code for code in df.Code.unique() if 'GWA_' in code]
df = df[df.Code.isin(gwa_codes)]

### Unstack the dataset

Pivot the dataset while keeping only one price column

In [8]:
pivoted_df = df.pivot(index='Date', columns='Code', values='VWAP')
pivoted_df.tail()

Code         GWA_BTC  GWA_ETH  GWA_LTC  GWA_XLM  GWA_XRP
Date                                                    
2018-01-19 11,826.36 1,068.45   195.00     0.51     1.82
2018-01-20 13,062.68 1,158.71   207.58     0.52     1.75
2018-01-21 12,326.23 1,108.90   197.36     0.48     1.55
2018-01-22 11,397.52 1,038.21   184.92     0.47     1.43
2018-01-23 10,921.00   992.05   176.95     0.47     1.42

### Shift the dataset

Generate returns for 7, 14, 21, 28 day windows by shifting the pivoted dataset by the required number of days.

Assumptions: records are sorted ascending by date and there are no missing dates

In [11]:
delta_dict = {}
for offset in [7, 14, 21, 28]:
    delta_dict['delta_{}'.format(offset)] = pivoted_df / pivoted_df.shift(offset) - 1.0

In [13]:
delta_dict.keys()

dict_keys(['delta_7', 'delta_14', 'delta_21', 'delta_28'])

### Melt the shifted dataset

Unpivot to such that each row contains only info for a particular coin on a particular date

In [15]:
melted_7 = delta_dict['delta_7'].reset_index().melt(id_vars=['Date'], value_name='delta_7')
melted_7.tail()

Date     Code  delta_7
6965  2018-01-19  GWA_XRP    -0.22
6966  2018-01-20  GWA_XRP    -0.29
6967  2018-01-21  GWA_XRP    -0.30
6968  2018-01-22  GWA_XRP    -0.32
6969  2018-01-23  GWA_XRP    -0.04

In [16]:
melted_dfs = []
for key, delta_df in delta_dict.items():
    melted_dfs.append( delta_df.reset_index().melt(id_vars=['Date'], value_name=key) )

### Create target variable

Create another melted dataframe that contains the forward-looking 7-day returns.

Shift the pivoted dataset by -7  to get "future" prices

In [17]:
return_df = pivoted_df.shift(-7) / pivoted_df - 1.0
melted_dfs.append( return_df.reset_index().melt(id_vars=['Date'], value_name='return_7') )